In [1]:
import pandas as pd
from pathlib import Path

DATA_GOLD = Path("data/gold")

features = pd.read_csv(DATA_GOLD / "pokemon_power_score.csv")
clusters = pd.read_csv(DATA_GOLD / "pokemon_clusters_roles.csv")
type_off_rank = pd.read_csv(DATA_GOLD / "type_offensive_ranking.csv")
type_def_rank = pd.read_csv(DATA_GOLD / "type_defensive_ranking.csv")

features.shape, clusters.shape, type_off_rank.shape, type_def_rank.shape

((1328, 32), (1328, 22), (21, 2), (21, 2))

In [2]:
# Merge por seguridad (si no coinciden todas las columnas)
df = features.merge(
    clusters[["id", "role_kmeans"]],
    on="id",
    how="left"
)

top_offensive = (
    df
    .sort_values("offensive_score", ascending=False)
    .head(10)
    [[
        "id", "name", "type_1", "type_2",
        "offensive_score", "defensive_score", "power_score_final",
        "role_kmeans"
    ]]
)

top_defensive = (
    df
    .sort_values("defensive_score", ascending=False)
    .head(10)
    [[
        "id", "name", "type_1", "type_2",
        "offensive_score", "defensive_score", "power_score_final",
        "role_kmeans"
    ]]
)

top_overall = (
    df
    .sort_values("power_score_final", ascending=False)
    .head(10)
    [[
        "id", "name", "type_1", "type_2",
        "offensive_score", "defensive_score", "power_score_final",
        "role_kmeans"
    ]]
)

top_offensive, top_defensive, top_overall

(         id                name  type_1  type_2  offensive_score  \
 1099  10075        diancie-mega    rock   fairy         0.825789   
 376     377            regirock    rock     NaN         0.807895   
 644     645  landorus-incarnate  ground  flying         0.804211   
 1045  10021    landorus-therian  ground  flying         0.804211   
 1111  10087       camerupt-mega    fire  ground         0.801579   
 1096  10072        steelix-mega   steel  ground         0.797368   
 718     719             diancie    rock   fairy         0.794211   
 75       76               golem    rock  ground         0.791053   
 377     378              regice     ice     NaN         0.787895   
 1098  10074         glalie-mega     ice     NaN         0.787895   
 
       defensive_score  power_score_final      role_kmeans  
 1099         0.379576           0.647304  Powerhouse Tank  
 376          0.408824           0.648266     Glass Cannon  
 644          0.347703           0.621607     Glass Cann

In [3]:
df["power_per_exp"] = df["power_score_final"] / df["base_experience"]

value_pokemon = (
    df[df["base_experience"] > 0]
    .sort_values("power_per_exp", ascending=False)
    .head(10)
    [[
        "id", "name", "type_1", "type_2",
        "base_experience", "power_score_final", "power_per_exp",
        "role_kmeans"
    ]]
)

value_pokemon

,id,name,type_1,type_2,base_experience,power_score_final,power_per_exp,role_kmeans
836,837,rolycoly,rock,NaN,48.0,0.450065,0.009376,Glass Cannon
523,524,roggenrola,rock,NaN,56.0,0.475093,0.008484,Glass Cannon
437,438,bonsly,rock,NaN,58.0,0.486324,0.008385,Glass Cannon
272,273,seedot,grass,NaN,44.0,0.367846,0.008360,Support / Utility
73,74,geodude,rock,ground,60.0,0.483575,0.008060,Glass Cannon
217,218,slugma,fire,NaN,50.0,0.383830,0.007677,Support / Utility
342,343,baltoy,ground,psychic,60.0,0.458251,0.007638,Glass Cannon
581,582,vanillite,ice,NaN,61.0,0.465313,0.007628,Glass Cannon
596,597,ferroseed,grass,steel,61.0,0.463871,0.007604,Glass Cannon
89,90,shellder,water,NaN,61.0,0.463757,0.007603,Glass Cannon


In [4]:
# Top 5 ofensivos
top_types_off = type_off_rank.sort_values(
    "expected_offensive_multiplier", ascending=False
).head(5)

# Peores 5 ofensivos
worst_types_off = type_off_rank.sort_values(
    "expected_offensive_multiplier", ascending=True
).head(5)

# Top 5 defensivos (menor daño recibido)
top_types_def = type_def_rank.sort_values(
    "expected_defensive_multiplier", ascending=True
).head(5)

top_types_off, worst_types_off, top_types_def

(  attacking_type  expected_offensive_multiplier
 0         ground                       1.142857
 1           rock                       1.119048
 2           fire                       1.095238
 3            ice                       1.095238
 4         flying                       1.071429,
    attacking_type  expected_offensive_multiplier
 20         normal                       0.904762
 19         poison                       0.952381
 18            bug                       0.976190
 17         dragon                       0.976190
 16       electric                       0.976190,
   defending_type  expected_defensive_multiplier
 0          steel                       0.857143
 1          ghost                       0.952381
 2       electric                       0.976190
 3          fairy                       0.976190
 4         poison                       0.976190)

In [5]:
roles_count = df["role_kmeans"].value_counts().reset_index()
roles_count.columns = ["role", "n_pokemon"]
roles_count

,role,n_pokemon
0,Bruiser / All-Rounder,571
1,Support / Utility,394
2,Powerhouse Tank,231
3,Glass Cannon,132


In [6]:
examples_by_role = {}

for role in df["role_kmeans"].dropna().unique():
    examples_by_role[role] = (
        df[df["role_kmeans"] == role]
        .sort_values("power_score_final", ascending=False)
        [["name", "type_1", "type_2", "offensive_score", "defensive_score", "power_score_final"]]
        .head(5)
    )

examples_by_role

{'Support / Utility':           name  type_1  type_2  offensive_score  defensive_score  \
 621     golett  ground   ghost         0.590421         0.138833   
 535  palpitoad   water  ground         0.536000         0.197311   
 230     phanpy  ground     NaN         0.548947         0.171394   
 103     cubone  ground     NaN         0.545789         0.171357   
 957  tinkatuff   fairy   steel         0.474737         0.262128   
 
      power_score_final  
 621           0.409786  
 535           0.400524  
 230           0.397926  
 103           0.396017  
 957           0.389693  ,
 'Bruiser / All-Rounder':            name  type_1  type_2  offensive_score  defensive_score  \
 804   stakataka    rock   steel         0.574737         0.418284   
 320     wailord   water     NaN         0.642632         0.311739   
 1002    ting-lu    dark  ground         0.574737         0.401286   
 449   hippowdon  ground     NaN         0.610526         0.343651   
 749    mudsdale  ground     Na

In [7]:
out_dir = DATA_GOLD / "exports_for_report"
out_dir.mkdir(parents=True, exist_ok=True)

top_offensive.to_csv(out_dir / "top10_offensive_for_report.csv", index=False)
top_defensive.to_csv(out_dir / "top10_defensive_for_report.csv", index=False)
top_overall.to_csv(out_dir / "top10_overall_for_report.csv", index=False)
value_pokemon.to_csv(out_dir / "top10_value_for_report.csv", index=False)
roles_count.to_csv(out_dir / "roles_counts_for_report.csv", index=False)

out_dir

PosixPath('data/gold/exports_for_report')